# ketos jupyter notebook machine learning example

Hi there, this is your personal model building environment. You have full access to your environment and can install any R packages you need and build your own models.

Rhis is an example for a ketos jupyter notebook, that will guide you through the model development and deployment process.

TRAIN YOUR MODEL

To train your model you can install any R packages you might need, add any additional functions and build your own model.

For any good building process you will need data, here is how you get it:

1. go to your ketos user interface and request a new dataset. Once your dataset has been prepared for you, you can access it from within this notebook, to do this just substitute the string 
"paste your training data url from your ketos system here to get your training data"
in the "ketos_train" cell with the URL you got.


DEPLOY YOUR MODEL

to deploy the notebook in order to allow for external predictions just do the following:

1. tag the cells that you would like to be executed for your prediction with the keyword "ketos_predict" in the first line of each cell as a comment.

2. tag the cells which produce the output for the prediction with the keyword "ketos_predict_output", please note that this cell will be executed and the output will be used as return value for the prediction. It is expected to be in json format as supplied by the "format_to_json" function provided here.


In [ ]:
# ketos_init
install.packages(paste(c('geoR', 'RandomFieldsUtils', 'msm')))

In [ ]:
ret_json = ""
for(i in c("2030", "3040", "4050", "5060", "6070", "7080")){
        hbData = read.csv(paste('/mlenvironment/models/mlmodel_1/prog/results/result', i, '_2.csv', sep=""))
        minAge = as.numeric(substring(i, 0, 2))
        maxAge = as.numeric(substring(i, 2, 4))
        mu = hbData[1,20]
        sigma = hbData[1,21]
        lambda = hbData[1,22]
        ret_json = paste(ret_json, '"age',i,'":', sep="")
    }

In [ ]:
# ketos_train

for(i in c("2030", "3040", "4050", "5060", "6070", "7080")){
    setwd('/mlenvironment/models/mlmodel_1/prog')
    arg1 <- paste('/mlenvironment/models/mlmodel_1/prog/data/par', i, '.csv', sep="")
    getwd()
    paste("Rscript R_program2_20180507.R",arg1)
    system(paste("Rscript /mlenvironment/models/mlmodel_1/prog/R_program2_20180507.R", arg1),intern = FALSE,
           ignore.stdout = FALSE, ignore.stderr = FALSE,
           wait = TRUE, input = NULL, timeout = 0)  
}

In [ ]:
#ketos_predict, ketos_predict_output
percentile = function(percentile, mu, sigma, lambda){
    
    if(lambda == 0){
        return (exp(qnorm(percentile, mean=mu, sd=sigma)))
    }else{
        return ((qnorm(percentile, mean=mu, sd=sigma)* lambda + 1) ^ (1/lambda))
    }
       
}


format_to_json_age_group = function(mu, sigma, lambda, minAge, maxAge){
    ret_json = "{"  
    ret_json = paste(ret_json,'"minAge":', minAge , sep="")
    ret_json = paste(ret_json,',"maxAge":', maxAge , sep="")
    ret_json = paste(ret_json,',"mu":', mu , sep="")
    ret_json = paste(ret_json,',"sigma":', sigma , sep="")
    ret_json = paste(ret_json,',"lambda":', lambda , sep="")
    ret_json = paste(ret_json,',"percentile2.5":', percentile(0.025, mu, sigma, lambda) , sep="")
    ret_json = paste(ret_json,',"percentile97.5":', percentile(0.975, mu,sigma, lambda) , sep="")
    ret_json = paste(ret_json,',"percentile50":', percentile(0.500, mu,sigma, lambda) , sep="")
    ret_json = paste(ret_json,'}',sep="")
    return (ret_json)   
}

format_to_json = function(){
    ret_json = "{"  
    
    ret_json = paste(ret_json, '"M":', '{', sep="")
    
    for(i in c("2030", "3040", "4050", "5060", "6070", "7080")){
        hbData = read.csv(paste('/mlenvironment/models/mlmodel_1/prog/results/result', i, '_2.csv', sep=""))
        minAge = as.numeric(substring(i, 0, 2))
        maxAge = as.numeric(substring(i, 2, 4))
        mu = hbData[1,20]
        sigma = hbData[1,21]
        lambda = hbData[1,22]
        
        #print(paste("age =", i," lambda = ",lambda , sep=""))
        if(i == "2030"){
            ret_json = paste(ret_json, '"age',i,'":', format_to_json_age_group(mu,sigma, lambda, minAge, maxAge), sep="")
        }else{
            ret_json = paste(ret_json, ',"age',i,'":', format_to_json_age_group(mu,sigma, lambda, minAge, maxAge), sep="")
        }
    }
    
    ret_json = paste(ret_json, '},', sep="")
    
    ret_json = paste(ret_json, '"F":', '{', sep="")
    for(i in c("2030", "3040", "4050", "5060", "6070", "7080")){
        hbData = read.csv(paste('/mlenvironment/models/mlmodel_1/prog/results/result', i, '_2.csv', sep=""))
        minAge = as.numeric(substring(i, 0, 2))
        maxAge = as.numeric(substring(i, 2, 4))
        mu = hbData[2,20]
        sigma = hbData[2,21]
        lambda = hbData[2,22]
        
        if(i == "2030"){
            ret_json = paste(ret_json, '"age',i,'":', format_to_json_age_group(mu,sigma, lambda, minAge, maxAge), sep="")
        }else{
            ret_json = paste(ret_json, ',"age',i,'":', format_to_json_age_group(mu,sigma, lambda, minAge, maxAge), sep="")
        }
    }
    
    ret_json = paste(ret_json, '}', sep="")
    
    ret_json = paste(ret_json, "}", sep="")

    return (ret_json)
}

ketos_predict = function(){
    # ketos_predict_data = ("csv url will be inserted for the predict call here automatically - paste your url here to test ur predict function") 
    
    prediction = format_to_json()
    return (prediction)
}

print(ketos_predict(), quote=FALSE)

